##  Unsupervised Learning Series

# 9.4  Recommender Systems

In this Section we discuss the fundamental linear *Recommender System*, a popular unsupervised learning framework commonly employed by businesses to help automatically recommend products and services to their customers.  From the vantage of machine learning however, the basic Recommender System detailed here is simply a slight twist on  our core unsupervised learning technique: Principal Component Analysis (PCA).  

## 9.4.1 Introduction and motivation

Recommender systems are heavily used in e-commerce today, providing customers with personalized recommendations for products and services by using a consumer's previous purchasing and rating history, along with those of similar customers.  For instance, a movie provider like Netflix with millions of users and tens of thousands of movies, records users' reviews and ratings (typically in the form of a number on a scale of 1-5 with 5 being the most favorable rating) in a large matrix such as the one illustrated below in the Figure. These matrices are very sparsely populated, since an individual consumer has likely rated only a small portion of the movies available. With this data available, online movie and commerce sites often use the unsupervised learning technique we discuss in this Section as their main technique for making personalized recommendations to customers regarding what they might like to watch / consume next. With the technique for producing personalized recommendations we discuss here - typically referred to as a *Recommender System* - we aim to intelligently guess the values of every missing entry in the ratings matrix (we *complete* the matrix).  Then, in order to recommend a new product to a given user, we examine our completely filled ratings matrix for products we have predicted the user would highly rate (and thus enjoy) and recommend these.  

<figure>
  <img src= '../../mlrefined_images/unsupervised_images/Fig_9_11.png' width="100%" height="auto" alt=""/>
  <figcaption>   
<strong>Figure 1:</strong> <em> A prototypical movie rating matrix is very sparsely populated, with each user having rated only a very small number of films. In this diagram movies are listed along rows with users along columns.  In order to properly recommend movies for users to watch we try to intelligently guess the missing values of this matrix, and then recommend movies that we predict users would highly rate (and therefore enjoy the most).
  </em>  </figcaption> 
</figure>

In [33]:
# This code cell will not be shown in the HTML version of this notebook
# imports from custom library
import sys
sys.path.append('../../')
import matplotlib.pyplot as plt
import numpy as np
import copy

# custom libs
from mlrefined_libraries import unsupervised_library as unsuplib
from mlrefined_libraries import basics_library as baslib
datapath = '../../mlrefined_datasets/unsuperlearn_datasets/'

# this is needed to compensate for matplotlib notebook's tendancy to blow up images when plotted inline
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 9.4.2  Formal modeling

In what follows we will use the familiar notation $\mathbf{x}_1,...,\mathbf{x}_P$ to denote our input data each of dimension $N$ - here $\mathbf{x}_p$ is our $p^{th}$ user's rating vector each of which is sparsely populated.  Formally we can express the index set of entries of these ratings vectors we actually have as

\begin{equation}
\Omega = \left \{\,\,\left(p,j\right)\,\,\rvert \,\,x_{p,\,j}  \,\,\text{exists}  \right \}.
\end{equation}

Stacking these user-rating vectors columnwise gives us our ratings matrix that we wish to complete

\begin{equation}
\mathbf{X} = \begin{bmatrix} 
\vert \,\,\,\,\,\, \vert \,\,\,\,\,...\,\,\,\,\vert \\
\mathbf{x}_1 \,\, \mathbf{x}_2 \,\,...\,\,\mathbf{x}_P \\
\vert \,\,\,\,\,\, \vert \,\,\,\,\,...\,\,\,\,\vert
\end{bmatrix}
\end{equation}

In order to complete a sparsely populated ratings matrix effectively - because the data we are aiming to fill in is literally missing - we have no choice but to make assumptions about how users' tastes behave in general.   The most common / simplest assumption we can make - and the one we discuss here - is that every user's tastes can be expressed as a linear combination of some small set of fundamental user taste-profiles.  For example, in the case of movies these profiles could include the prototypical romance movie lover, prototypical comedy movie lover, action movie lover, etc.,  The relatively small number of such categories or user types compared to the total number of users or movies / products / etc., in a ratings matrix, provides a useful framework to intelligenty guess at the ratings matrix's missing values. 

How does this assumption help us complete the ratings matrix?  In order to find out we need to translate this intuitive assumption into mathematics.  When stated mathematically this assumption says that there are some set of $K < N$ prototypical user-rating profile basis vectors $\mathbf{c}_1,\,\mathbf{c}_2,\,..,\,\mathbf{c}_K$ so that we can express (approximately) every *complete* user-profile vector (given also the ideal weights in each linear combination) as 

\begin{equation}
\sum_{n=1}^K \mathbf{c}_n w_{p,n} \approx \mathbf{x}_p  \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\, p=1...P
\end{equation}

Again here we suppose that both the prototypical rating profile vectors $\mathbf{c}_1,\,...\,\mathbf{c}_K$ and the weights $w_{p,n}$ are ideal.  In practice we need to learn the proper values of these parameters, and from here we could then propose to square the difference of each desired approximation above and sum the result (which would give precisely the Least Squares cost function for PCA we derived in Section 9.2.1 and also 9.2.5).  

However before doing this here we need to be careful - remember we only have access to entries of the ratings matrix whose indices lie in the set $\Omega$.  On the other hand, the ideal approximation above gives us an estimate for *every* entry of the matrix.  Before forming a Least Squares cost function whose minimizer corresponds to properly tuned parameters here we need to restrict the ideal approximations above to the set of data we actually have (those entries in our index set $\Omega$).  We can write this set of actual equalities somewhat abstractly by using the notation $\approx_{\Omega}$ to denote an approximation only for indices in the set $\Omega$ giving the similar looking formula

\begin{equation}
\sum_{n=1}^K \mathbf{c}_n w_{p,n} \approx_{\Omega} \mathbf{x}_p    \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\, p=1...P.
\end{equation}

In other words, we only care about the approximation holding for indecies in the set $\Omega$ (since these are the only non-empty entries of $\mathbf{X}$ / ratings entries we have).

BLAH  average over all $P$ points gives a Least Squares cost function for learning recommendations that closely mirrors the one or PCA as

\begin{equation}
g\left(\mathbf{w}_1,...,\mathbf{w}_p,\mathbf{c}_1,...,\mathbf{c}_K\right) = \frac{1}{P}\sum_{p=1}^P \left.\left(\sum_{n=1}^K \mathbf{c}_n w_{p,n} - \mathbf{x}_p   \right)^2\right\vert_{\Omega}.
\end{equation}

Since it is indeed so similar to the cost function for PCA it should be inticing - at least at first - to wonder whether or not we can divine a straight-forward minimizer if we assume additionally that the set of $K$ spoanning vectors is *orthonormal* (since we found that enforcing this constraint similarly produced a stunningly simple solution to PCA).  Unfortunately the fact that 

## 9.4.3  Optimization

A *block-coordinate descent* approach is a very natural approach to minimizing this cost function (much as it is with the analgous PCA cost discussed in Section 9.2.1 and 9.2.5).  By cycling through each vector of parameters $\mathbf{w}_1,...,\mathbf{w}_P,\mathbf{c}^{1},...,\mathbf{c}^{\,n}$ we can easily solve the first order system in each independently (keeping all of the others fixed), and hence sequentially minimize the cost function.  One can easily check that first order system of the cost function above in $\mathbf{w}_p$ alone (keeping all other variables fixed) is 

\begin{equation}
\left(\sum_{(\cdot,\,j)\in \Omega}\mathbf{c}^{\,j}\left(\mathbf{c}^{\,j}\right)^T\right)\,\mathbf{w}_p = \sum_{(\cdot,\,j)\in \Omega}x_{p,\,j}\left(\mathbf{c}^{\,j}\right)^T.
\end{equation}

This is an $N\times N$ linear and symmetric system of equations that is easily solvable (via e.g., coordinate descent in each individual entry of $\mathbf{w}_p$), whose solution gives the updated version of $\mathbf{w}_p$.  

The analgous first order system for $\mathbf{c}^{\,j}$ can likewise be computed as 

\begin{equation}
\left(\sum_{(p,\cdot)\in \Omega}\mathbf{w}_p^{\,}\mathbf{w}_p^{T}\right) \left(\mathbf{c}^{\,j}\right)^T =  \sum_{(p,\,\cdot)\in \Omega}x_{p,\,j}\mathbf{w}_p.
\end{equation}

This is a $K\times K$ linear and symmetric system of equations that is easily solvable (via e.g., coordinate descent in each individual entry of $\mathbf{c}^{\,j}$).

Sweeping through the parameter vectors and performing these updates a number of times results in a proper solution to the Least Squares cost function detailed above.  Below we collect these update steps in a pseudo-code block for convenience.  Comparing this procedure to the block-coordinate descent algorithm described in Section 9.2.1, notice that if $\Omega = \emptyset$, that is if we have every rating, then we are performing PCA here.  

### Recommender systems  (block-coordinate descent) 

<hr style="height:1px;border:none;color:#555;background-color:#555;">
<p style="line-height: 1.7;">
<strong>1:</strong>&nbsp;&nbsp; <strong>input:</strong> a number $K \leq N$ of desired principal components, dataset $\mathbf{x}_1,...,\mathbf{x}_P$, initializations for basis $\mathbf{c}_1,...,\mathbf{c}_K$, and maximum number of iterations $\text{max_its}$ <br>

<strong>2:</strong>&nbsp;&nbsp; <code>compute</code> mean of dataset $\boldsymbol{\mu} = \frac{1}{P}\sum_{p=1}^P\mathbf{x}_p$ and center data as $\mathbf{x}_p \longleftarrow \mathbf{x}_p - \boldsymbol{\mu}$ for $p=1,...,P$ <br> 

<strong>3:</strong>&nbsp;&nbsp; <code>for</code> $\,\,i = 1,\ldots,\text{max_its}$<br>

<strong>4:</strong>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <code># Update weight vectors</code><br>

<strong>5:</strong>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <code>for</code> $\,\,p = 1,\ldots,P$<br>

<strong>6:</strong>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <code> solve </code> $\mathbf{C}^T\mathbf{C}^{\,}\mathbf{w}_p = \mathbf{C}^T\mathbf{x}_p$ for $\mathbf{w}_p$ <br>

<strong>7:</strong>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <code>end for</code><br>

<strong>8:</strong>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <code># Update basis</code><br>

<strong>9:</strong>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <code>for</code> $\,\,n = 1,\ldots,K$<br>

<strong>10:</strong>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <code> solve </code> $\mathbf{c}_n = \frac{\sum_{p=1}^P \mathbf{x}_p w_{p,n} } {\sum_{p=1}^P w_{p,n}^2}$<br>

<strong>11:</strong>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <code>end for</code><br>

<strong>12:</strong>&nbsp; <code>end for</code><br>

<strong>13:</strong>&nbsp;&nbsp; <code># Update weights on final basis</code><br>


<strong>14:</strong>&nbsp;&nbsp;&nbsp; <code>for</code> $\,\,p = 1,\ldots,P$<br>

<strong>15:</strong>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <code> solve </code> $\mathbf{C}^T\mathbf{C}^{\,}\mathbf{w}_p = \mathbf{C}^T\mathbf{x}_p$ for $\mathbf{w}_p$ <br>

<strong>16:</strong>&nbsp;&nbsp; <code>end for</code><br>

<strong>17:</strong>&nbsp; <strong>output:</strong> optimal PCA basis $\mathbf{c}_1,...,\mathbf{c}_K$ and weights $\mathbf{w}_1,...,\mathbf{w}_P$<br>

<hr style="height:1px;border:none;color:#555;background-color:#555;">
</p>

#### <span style="color:#a50e3e;">Example 1: </span>  A simple example learning a spanning set via Principal Component Analysis 

In [ ]:
def update_weights(X,C):
    # Update weight vectors
    P,N = X.shape
    W = []
    
    # pre-compute elements of each linear system
    CTC = np.dot(C.T,C)
    for p in range(N):
        # setup linear system
        w_p = np.array(np.linalg.solve(CTC,np.dot(C.T,X[:,p][:,np.newaxis]))).ravel()
        W.append(w_p)
        
    # return weight matrix
    W = np.array(W).T
    return W

def update_basis(X,W):
    # Update basis vectors 
    K,P = W.shape
    C = []
    for n in range(K):
        # update nth element of basis
        c_n = np.sum(X*W[n,:][np.newaxis,:],axis = 1)/(np.sum(W[n,:]*W[n,:]))
        C.append(c_n)
        
    # return updated basis
    C = np.array(C).T
    return C

def block_coord_PCA(X,C,max_its):
    # Outer loop - over iterations
    for i in range(max_its):
        # update weights
        W = update_weights(X,C)

        # update basis
        C = update_basis(X,W)
        
    # final weight update
    W = update_weights(X,C)
    
    # return PCs and weights
    return C,W

In [20]:
def update_basis(X,W):
    # Update basis vectors 
    K,P = W.shape
    C = []
    for j in range(K):
        # setup linear system
        ind = np.isnan(X[j,:])
        om = np.argwhere(ind == True)
        A = np.dot(W[:,om],W[:,om].T)
        b =  np.sum(X[p,om]*X[:,om])

        # solve linear system and store
        c_j = np.linalg.lstsq(A,b).T 
        C.append(c_j)
        
    # return updated basis
    return C

def recommender_system(X,C,max_its):
    # Outer loop - over iterations
    for i in range(max_iters):
        # update weights
        W = update_weights(X,C)
        
        # update basis
        C = update_basis(X,W)
        
    # final weight update
    W = update_weights(X,C)

In [44]:
### Test matrix ###
# generate full ratings matrix
N = 10; K = 4;
X_orig = np.round(5*np.random.rand(N,N))
X = copy.deepcopy(X_orig)

# remove percentage of entries
removal_percentage = 0.5
removal_portion = round(removal_percentage*np.size(X_orig))
indices = np.random.permutation(np.size(X_orig))[:removal_portion]
X.ravel()[indices] = np.nan

In [45]:
# generate initialization for basis
C = np.random.randn(N,K)

In [63]:
def update_weights(X,C):
    # Update weight vectors
    N,P = X.shape
    W = []
    for p in range(P):
        # setup linear system
        ind = np.isnan(X[:,p])
        om = np.argwhere(ind == True).ravel()
        A = np.dot(C[om,:],C[om,:].T)
    
        
        b =  np.dot(X[om,p][np.newaxis,:],C[om,:])

        # solve linear system and store weights
#         print (np.shape(A))
#         print (np.shape(b))
        w_p = np.linalg.lstsq(A,b)
        W.append(w_p)
    
    # return weight matrix
    return np.array(W)

In [64]:
W = update_weights(X,C)

(4,)
(4, 4)
(1, 4)


LinAlgError: Incompatible dimensions